In [51]:
import urllib
import requests
import pandas as pd

API_KEY  = 'ENTER KEY'
df = pd.read_csv('SEZ.csv')

In [52]:
cache = {}
def getCoord(name):
    if name not in cache:
        LOCATION = name

        PARAMS = {'key':API_KEY , 'input':LOCATION , 'inputtype':'textquery','fields':'formatted_address,name,geometry'  }

        params_str = urllib.parse.urlencode(PARAMS)
        API_QUERY = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?'+params_str

        result = requests.get(url = API_QUERY) 

        # extracting data in json format 
        data = result.json()
        if(data['status'] == 'OK'):
            location = data['candidates'][0]['geometry']['location']
            cache[name] = (location['lat'],location['lng']) 
        else:
            cache[name] = (-1,-1)
    return cache[name]

In [53]:
getCoord('RV College')

(12.9237228, 77.4987012)

In [61]:
df['lat'] = -1
df['lng'] = -1
for index,row in df.iterrows():
#     print(str(row['SEZ Name']) + ', '+ str(row['Location']) )
    df['lat'].iat[index], df['lng'].iat[index] = getCoord(str(row['SEZ Name']) + ', '+ str(row['Location']))
    if(df.iloc[index]['lat'] == -1):
            df['lat'].iat[index], df['lng'].iat[index] = getCoord(str(row['SEZ Name']))

    print(str(row['SEZ Name']) + ', '+ str(row['Location']),df.iloc[index]['lat'])
    
print(df.iloc[0])

Kiadb Aerospace SEZ, nan 13
M/S. Apiic Ltd., Adibatla, Ibrahim Patnam, R .R. Dist,, Ranga Reddy District, Ap -1
Kinfra Film & Video Park (Kfvp), Trivandrum, Kerala 8
Mahindra, Chengleput -1
M/S Gmr Hyderabad Aviation SEZ Limited, Hyderbad, Village Mamidipalli, Rr District 17
Serum Bio Pharma Park SEZ, 212/2, Off Soil Poonawala Rod, Hadapsar, Pune -1
Apiic Ltd, Shameerpet Rr District -1
Biocon Special Economic Zone, Anekal Taluk, Bengaluru, Karnataka 12
Jubilant Infrastructure Ltd., Vagra, Bharuch 21
Wockhardt Infrastructure Development Ltd., Shendre Industrial Area, District Aurangabad -1
Maharashtra Industrial Development Corporation SEZ – Nanded, Krushnoor Industrial Area, Nanded District Nanded -1
Infosys Limited, Rajiv Gandhi Infotech Park, Ph.II, Village Mann, Taluka Mulashi, Pune 18
Wipro Ltd., Maharashtra 19
Neopro Technologies Pvt.Ltd SEZ (Formerly Known As M/S. Flagship Infrastructure Pvt. Ltd.), Village Hinjewadi Taluka Mulshi, Pune 18
The Manjri Stud Farm Pvt.Ltd, S.No.209 n

AMRL SEZ, Nanguneri Taluk, Tirunelvelli District 8
Visakhapatnam Special Economic Zone, Visakhapatnam 17
Andhra Pradesh Special Economic Zone, Atchuthapuram, Visakhapatnam 17
M/S. Apiic Ltd., Naidupeta, Nellore, Ap 13
Sricity Pvt. Ltd., Chittoor, Chittoor 13
Iffco Kisan SEZ, Nellore, A.P 14
Mangalore Special Economic Zone, Karnataka 12
Cochin Special Economic Zone, Cochin, Kerala 10
Indore SEZ, Sector-3, Pithampur Distt. Dhar (Mp) 22
Maharashtra Airport Development Company Ltd. ( Mihan SEZ ), Mihan, District Nagpur 21
Noida Special Economic Zone, Uttar Pradesh 28
Falta Special Economic Zone, Falta, West Bengal 22
New Chennai, Cheyyur 12
Kandla Special Economic Zone, KaSEZ, Kutch 23
Surat Special Economic Zone, Sachin, Surat 21
Adani Mundra Port, Mundra 22
Dahej SEZ Ltd., (Pertains To DC, Dahej SEZ) 21
Reliance Jamnagar Infrastructure Ltd., Jamnagar 22
Sterling SEZ Pvt.Ltd., (Pertains To DC, Sterling Ez) 22
MEPZ Special Economic Zone, Chennai 12
Euro Multivision Pvt. Ltd., Vill. Shikra,